# Fine-tuning RadBert #
Full parameter fine tuning of RadBERT model (RadBERT-RoBERTa-4m model from the RadBERT paper). Fine tuning carried out on the multi-label muti-class classification of reports, where each report can have multiple labels (For ex, a report can havel label consolidation-right and consolidation-2)

In [1]:
import os
import datetime
import codecs
from itertools import product

import numpy as np
import pandas as pd

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

import transformers
from transformers import AutoTokenizer, AutoModel
from transformers import pipeline

/home/users/pranav.rao/miniconda3/envs/TutorialCuda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

torch.set_printoptions(linewidth=200)

In [3]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


## Forward pass ##
Implementing RadBERTMultiClassMulti Label PyTorch Model

In [27]:
class RadBERTMultiClassMultiLabel(nn.Module):
    """
    RadBERTMultiClassMultiLabel: Model expects batches of natural language sentences, will
    classify reports with multiple label
    """
    def __init__(self, num_classes, checkpoint, device):
        super().__init__()
        self.num_classes = num_classes
        self.checkpoint = checkpoint
        self.device = device

        self.tokenizer = AutoTokenizer.from_pretrained(self.checkpoint)
        self.transformer_encoder = AutoModel.from_pretrained(self.checkpoint)
        self.transformer_encoder_hidden_size = self.transformer_encoder.config.hidden_size
        self.linear_classifier = nn.Linear(self.transformer_encoder_hidden_size, self.num_classes)
    
    def forward(self, x):
        tokenized_inp = self.tokenizer(x, padding=True, truncation=True, return_tensors='pt').to(self.device)
        encoder_out = self.transformer_encoder(**tokenized_inp)
        logits = self.linear_classifier(encoder_out.last_hidden_state[:, 0, :])
        return logits


In [28]:
checkpoint = 'UCSD-VA-health/RadBERT-RoBERTa-4m'
labels_subset = "normal tuberculosis opacity bronchialdilation density parenchymalopacity ett aorticenlargement mediastinalwidening mediastinalmass\
        copd prominentbronchovascularmarkings bronchitis markings vascularprominence interval interstitiallungdisease bluntedcp effusion cardiomegaly\
        consolidation subtle_normal peffusion lineandtube thickening haziness hilarprominence hilar inhomogenousopacity rotation\
        calcification unfoldedaorta bandlikeopacity aorticcalcification aorticknucklecalcification fibrosis suture cardiacshift degenspine nodule\
        pneumonia inspiration fracture pneumonitis justfibrosis lesion nonaorticcalcification tuberculosispure pleuralthickening feedingtube".split()
num_classes = len(labels_subset)

In [29]:
radbert_multi_model = RadBERTMultiClassMultiLabel(num_classes, checkpoint, device).to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at UCSD-VA-health/RadBERT-RoBERTa-4m and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
print(radbert_multi_model)
print(list(map(lambda x : x.shape, radbert_multi_model.parameters())))

RadBERTMultiClassMultiLabel(
  (transformer_encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
      

## Custom Loss function ##
Custom loss function for multi class multi label classification to handle uncertain tags (tags have value 0 -> absent, 1 -> present, -100 -> uncertain)

In [31]:
class MultiClassMultiLabel(nn.Module):
    def __init__(self, uncertain_label):
        super(MultiClassMultiLabel, self).__init__()
        self.uncertain_label = uncertain_label
    
    def forward(self, output, target):
        certain_mask = (target != self.uncertain_label)
        loss_func = nn.MultiLabelSoftMarginLoss(weight=certain_mask.type(torch.float))
        return loss_func(output, target)

In [32]:
multiclass_multilabel_loss = MultiClassMultiLabel(-100).to(device)

### Testing MultiClassMultiLabel loss function ###

In [34]:
logit_tensor = torch.Tensor([[-1.0, 2.0, 1.0, 5.0, -3.0], [4.0, -2.0, 1.0, -1.0, 2.5]]).to(device)
target_tensor_act = torch.Tensor([[0, 1, -100, 0, 0], [1, -100, 0, 0, 1]]).to(device)

In [35]:
#certain_mask = (target_tensor_act != -100)
#print(certain_mask.type(torch.float))
#loss_func = nn.MultiLabelSoftMarginLoss(weight=(certain_mask).type(torch.float))
#print(loss_func(logit_tensor, target_tensor_act))
print(multiclass_multilabel_loss(logit_tensor, target_tensor_act))

tensor(0.7219, device='cuda:0')


In [36]:
import math
from math import log, exp

def log_sigmoid(x):
    return log(1/(1+exp(-1*x)))

In [37]:
loss1 = -1 * (1.0 * log_sigmoid(1.0) + 1.0 * log_sigmoid(2.0) + 0.0 * log_sigmoid(-1.0) + 1.0 * log_sigmoid(-5.0) + 1.0 * log_sigmoid(3.0)) / 5.0
loss2 = -1 * (1.0 * log_sigmoid(4.0) + 0.0 * log_sigmoid(2.0) + 1.0 * log_sigmoid(-1.0) + 1.0 * log_sigmoid(1.0) + 1.0 * log_sigmoid(2.5)) / 5.0

print(loss1)
print(loss2)
print((loss1 + loss2)/2.0)

1.0990984797248111
0.34471260744936094
0.721905543587086


## Custom DataLoader and Dataset ##
Read csv file, get the report from path and prepare the data

In [38]:
class ReportTagsDataset(Dataset):
    def __init__(self, tags_csv_file, report_base_path, labels_subset=None, text_transform=None, target_transform=None):
        self.report_base_path = report_base_path
        self.tags_csv_file = tags_csv_file

        self.tags_df = pd.read_csv(self.tags_csv_file)
        self.column_names = list(self.tags_df.columns.values)
        self.column_names[0] = 'filename'
        self.tags_df.columns = self.column_names

        self.labels_subset = labels_subset
        self.text_transform = text_transform
        self.target_transform = target_transform
    
    def __len__(self):
        return self.tags_df.shape[0]
    
    def __getitem__(self, index):
        report_path = os.path.join(self.report_base_path, self.tags_df.iloc[index, 0].split('/')[-1] + '.txt')
        #report_text = open(report_path).read()
        report_text = codecs.open(report_path, 'r', encoding='utf-8', errors='ignore').read()
        if self.labels_subset is None:
            target_list = torch.Tensor(list(self.tags_df.iloc[index][1:]))
        else:
            target_list = torch.Tensor(list(self.tags_df[self.labels_subset].iloc[index]))
        return report_text, target_list

In [39]:
#report_base_path = "/models_common_e2e/cxr_data/reports/training"
train_reports_base_path = '/home/users/pranav.rao/MiniTasks/Radbert/data/train'
test_reports_base_path = '/home/users/pranav.rao/MiniTasks/Radbert/data/test'
train_tags_file = '/home/users/pranav.rao/Downloads/report_tags_25k_train.csv'
test_tags_file = '/home/users/pranav.rao/Downloads/report_tags_25k_train.csv'

In [40]:
train_data = ReportTagsDataset(train_tags_file, train_reports_base_path, labels_subset=labels_subset)
test_data = ReportTagsDataset(test_tags_file, test_reports_base_path, labels_subset=labels_subset)

In [51]:
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

### Testing Dataset and DataLoader ###

In [ ]:
# Display image and label.
train_features, train_labels = next(iter(train_dataloader))

In [43]:
print(train_features)
print(train_labels)
print(f"Feature batch shape: {len(train_features)}")
print(f"Labels batch shape: {len(train_labels)}")
report_text = train_features[0]
print(report_text)
label = train_labels[0]
print(f"Label: {label}")

('XR- CHEST PA VIEW\n\nFINDINGS:\n\nLUNGS: Normal.\nTRACHEA: Normal. \nCARINA: Normal. \nRIGHT AND LEFT MAIN BRONCHI: Normal. \nPLEURA: Normal. \n\nHEART: Normal. \nRIGHT HEART BORDER: Normal. \nLEFT HEART BORDER: Normal. \nPULMONARY BAY: Normal. \nPULMONARY HILA: Normal. \nAORTA: Normal. \n\nTHORACIC SPINE: Normal.\nOTHER VISUALIZED BONES: Normal. \nVISUALIZED SOFT TISSUES: Normal. \n\nDIAPHRAGM: Normal. \n\nVISUALIZED ABDOMEN: Normal. \nVISUALIZED NECK: Normal.', 'Patient ID 6f6c6ca33ee7bd64f140633025a4051e86dfbe7985bcf9f2ba4c245a\r\nPatient Name ccd620b0c42218c3fa6b1067d18694eb7f777d1cf40b2c498e098274 51Y\r\nAge :\r\nSex :F\r\nAccession Number 21f8861d6df137bb07d47c0fec31dad975e4f89c5ba5c163e04c4cf9\r\nModality :CR\r\nReferring Physician 21f8861d6df137bb07d47c0fec31dad975e4f89c5ba5c163e04c4cf9\r\nStudy :CHEST\r\nStudy Date 1741ca7c811cee8c3691fce71e91411d170009d920b58244a4d63e4c\r\n\r\n\r\n\r\n\r\n\r\nX-RAY CHEST PA VIEW\r\n\r\n\r\nThe lungs on the either side show equal translucenc

In [ ]:
logit_tensor = radbert_multi_model(train_features)

In [ ]:
print(logit_tensor)
print(logit_tensor.shape)

tensor([[-0.1485, -0.3681,  0.0774,  ..., -0.2707, -0.0061, -0.1722],
        [-0.1415, -0.3708,  0.1239,  ..., -0.2729, -0.0105, -0.1739],
        [-0.1658, -0.3611,  0.0246,  ..., -0.2912,  0.0304, -0.1404],
        ...,
        [-0.1485, -0.3477,  0.0417,  ..., -0.2952,  0.0045, -0.1395],
        [-0.1423, -0.3476,  0.0403,  ..., -0.3019, -0.0042, -0.1491],
        [-0.1514, -0.3525,  0.0412,  ..., -0.3050,  0.0014, -0.1509]], grad_fn=<AddmmBackward0>)
torch.Size([32, 322])


## Fine Tuning ##
Fine tuning the RadBERT model for tags prediction task

### Adam Optimizer ###
Using Adam optimizer with learning rate 3e-5, beta1 = 0.9, beta2 = 0.99, l2 weight decay of 0.01

In [45]:
lr = 3e-5
beta1 = 0.9
beta2 = 0.99
l2_weight_decay = 0.01

In [47]:
optimizer = torch.optim.Adam(radbert_multi_model.parameters(), lr=lr, betas=(beta1, beta2), weight_decay=l2_weight_decay)

In [48]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    for i, data in enumerate(train_dataloader):
        inputs, labels = data
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = radbert_multi_model(inputs)

        # Compute the loss and its gradients
        loss = multiclass_multilabel_loss(outputs, labels)
        loss.backward()
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 100 == 99:
            last_loss = running_loss / 100 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(train_dataloader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

### The full fine-tuning loog ###

In [49]:
total_epochs = 5

In [ ]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

for epoch in range(total_epochs):
    # Make sure gradient tracking is on, and do a pass over the data
    print('EPOCH {}:'.format(epoch_number + 1))
    radbert_multi_model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)

    # Set the model to evaluation mode, disabling dropout and using population, statistics for batch normalization
    running_vloss = 0.0
    radbert_multi_model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(test_dataloader):
            vinputs, vlabels = vdata
            vlabels = vlabels.to(device)
            voutputs = radbert_multi_model(vinputs)
            vloss = multiclass_multilabel_loss(voutputs, vlabels)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()
    model_path = '/home/users/pranav.rao/MiniTasks/Radbert/model_{}_{}'.format(timestamp, epoch_number)
    torch.save(radbert_multi_model.state_dict(), model_path)
    epoch_number += 1

## Analyzing fine-tuned models ##

In [5]:
sentence_list = ["The report shows small right-sided pleural effusion", "The report shows small left-sided pleural effusion",\
    "The report shows large right-sided pleural effusion", "The report shows large left-sided pleural effusion",\
    "There are no abnormalities in the report",\
    "There is severe consolidation in the left side","There is severe consolidation in the right side",\
    "There is mild consolidation in the right side", "There is mild consolidation in the left side"
]

sentence1_base = "A <SizeModifier> <AbnormalReport> can be seen in the report in the <LocationModifier> part"
sentence2_base = "The report shows a <SizeModifier> <LocationModifier> <AbnormalReport>"
size_modifiers = ['small', 'large']
loc_modifiers = ['upper-left', 'lower-left', 'right-sided', 'left-sided']
abnormal_report = ['pleural effusion']

l1 = [sentence1_base.replace('<SizeModifier>', size_mod).replace('<LocationModifier>', loc_mod).replace('<AbnormalReport>', ab_rep) for size_mod, loc_mod, ab_rep in product(size_modifiers, loc_modifiers, abnormal_report)]
l2 = [sentence2_base.replace('<SizeModifier>', size_mod).replace('<LocationModifier>', loc_mod).replace('<AbnormalReport>', ab_rep) for size_mod, loc_mod, ab_rep in product(size_modifiers, loc_modifiers, abnormal_report)]

negative_sentences = ['The report shows no pleural effusion', 'The report shows no consolidation on any side']
all_sentence_list = l1 + l2 + negative_sentences + sentence_list[4:]

In [6]:
print('\n'.join(all_sentence_list))

A small pleural effusion can be seen in the report in the upper-left part
A small pleural effusion can be seen in the report in the lower-left part
A small pleural effusion can be seen in the report in the right-sided part
A small pleural effusion can be seen in the report in the left-sided part
A large pleural effusion can be seen in the report in the upper-left part
A large pleural effusion can be seen in the report in the lower-left part
A large pleural effusion can be seen in the report in the right-sided part
A large pleural effusion can be seen in the report in the left-sided part
The report shows a small upper-left pleural effusion
The report shows a small lower-left pleural effusion
The report shows a small right-sided pleural effusion
The report shows a small left-sided pleural effusion
The report shows a large upper-left pleural effusion
The report shows a large lower-left pleural effusion
The report shows a large right-sided pleural effusion
The report shows a large left-sid

In [7]:
def calc_cosine_sim_matrix(sentence_embeddings):
    #stacked_sentence_embeddings = torch.stack(sentence_embeddings)
    stacked_sentence_embeddings = sentence_embeddings
    # Calculate the cosine similarity matrix
    cosine_sim_matrix = F.cosine_similarity(stacked_sentence_embeddings.unsqueeze(1), stacked_sentence_embeddings.unsqueeze(0), dim=2)
    return stacked_sentence_embeddings, cosine_sim_matrix

In [38]:
def get_sentence_embeddings(model, input):
    with torch.no_grad():
        tokenized_inp = model.tokenizer(input, padding=True, truncation=True, return_tensors='pt')
        encoder_out = model.transformer_encoder(**tokenized_inp)
    return encoder_out.last_hidden_state[:, 0, :]

In [69]:
model_path = '/home/users/pranav.rao/MiniTasks/Radbert/ModelPool/model_20230912_143355_0'
checkpoint = 'UCSD-VA-health/RadBERT-RoBERTa-4m'
radbert_multi_model = RadBERTMultiClassMultiLabel(num_classes, checkpoint)
radbert_multi_model.load_state_dict(torch.load(model_path))

Some weights of RobertaModel were not initialized from the model checkpoint at UCSD-VA-health/RadBERT-RoBERTa-4m and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [70]:
all_embeddings = get_sentence_embeddings(radbert_multi_model, all_sentence_list)

In [71]:
print(all_embeddings)
print(all_embeddings.shape)

tensor([[-0.2269,  0.5868, -0.1304,  ..., -2.0488, -0.0445,  0.8514],
        [-0.2324,  0.5791, -0.1244,  ..., -2.0475, -0.0269,  0.8427],
        [-0.2332,  0.5618, -0.1264,  ..., -2.0692, -0.0282,  0.8331],
        ...,
        [-0.3256,  0.5269, -0.2196,  ..., -2.1573,  0.0720,  0.7191],
        [-0.3190,  0.5626, -0.2126,  ..., -2.1307,  0.0728,  0.6843],
        [-0.3230,  0.5576, -0.2204,  ..., -2.1268,  0.0858,  0.6670]])
torch.Size([23, 768])


In [72]:
_, cosine_sim = calc_cosine_sim_matrix(all_embeddings)

In [73]:
print(cosine_sim)

tensor([[1.0000, 0.9999, 0.9997, 0.9998, 0.9999, 0.9998, 0.9996, 0.9997, 0.9982, 0.9983, 0.9976, 0.9977, 0.9979, 0.9981, 0.9973, 0.9974, 0.9937, 0.9922, 0.9903, 0.9940, 0.9938, 0.9939, 0.9940],
        [0.9999, 1.0000, 0.9997, 0.9998, 0.9997, 0.9999, 0.9996, 0.9997, 0.9979, 0.9983, 0.9975, 0.9975, 0.9975, 0.9980, 0.9971, 0.9972, 0.9933, 0.9919, 0.9902, 0.9937, 0.9935, 0.9937, 0.9938],
        [0.9997, 0.9997, 1.0000, 0.9999, 0.9996, 0.9997, 0.9999, 0.9998, 0.9982, 0.9984, 0.9982, 0.9981, 0.9978, 0.9982, 0.9979, 0.9978, 0.9940, 0.9925, 0.9906, 0.9944, 0.9945, 0.9946, 0.9945],
        [0.9998, 0.9998, 0.9999, 1.0000, 0.9997, 0.9997, 0.9998, 0.9999, 0.9982, 0.9984, 0.9981, 0.9981, 0.9978, 0.9982, 0.9977, 0.9978, 0.9940, 0.9924, 0.9905, 0.9944, 0.9942, 0.9943, 0.9945],
        [0.9999, 0.9997, 0.9996, 0.9997, 1.0000, 0.9999, 0.9998, 0.9998, 0.9983, 0.9983, 0.9977, 0.9977, 0.9982, 0.9984, 0.9977, 0.9977, 0.9941, 0.9924, 0.9905, 0.9942, 0.9941, 0.9940, 0.9941],
        [0.9998, 0.9999, 0.999

## Analysing the predictions ##

In [74]:
def get_predictions(model, input):
    with torch.no_grad():
        tokenized_inp = model.tokenizer(input, padding=True, truncation=True, return_tensors='pt')
        encoder_out = model.transformer_encoder(**tokenized_inp)
        logits = model.linear_classifier(encoder_out.last_hidden_state[:, 0, :])
        return logits

In [75]:
predictions = get_predictions(radbert_multi_model, all_sentence_list)

In [77]:
print(predictions)
print(predictions.shape)
print(predictions.argmax(dim=-1))
print(predictions[:, 224])

tensor([[-3.3179, -3.4485, -3.3018,  ..., -3.3865, -2.9830, -3.4156],
        [-3.3157, -3.4456, -3.3008,  ..., -3.3809, -2.9793, -3.4176],
        [-3.3140, -3.4461, -3.2968,  ..., -3.3888, -2.9813, -3.4179],
        ...,
        [-3.2344, -3.4549, -3.2411,  ..., -3.3429, -2.9685, -3.4108],
        [-3.2331, -3.4431, -3.2351,  ..., -3.3354, -2.9648, -3.4121],
        [-3.2290, -3.4323, -3.2309,  ..., -3.3308, -2.9543, -3.4143]])
torch.Size([23, 322])
tensor([224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224])
tensor([-0.4905, -0.4914, -0.4824, -0.4777, -0.4883, -0.4893, -0.4806, -0.4756, -0.4617, -0.4617, -0.4504, -0.4428, -0.4608, -0.4609, -0.4503, -0.4424, -0.4080, -0.4365, -0.4165, -0.4053, -0.4172,
        -0.4155, -0.4043])


In [78]:
df_tags = pd.read_csv('/home/users/pranav.rao/MiniTasks/Radbert/report_tags_25k_train.csv')

In [80]:
df_tags.describe()

       airspacedisease  perihilarairspacedisease  bronchiolitis  vascularprominence  vascularcongestion  perihilarcoarsening  hilarprominence   steeplesign  peribronchialthickening  interstitialthickening  interlobularseptalthickening  tortuousaorta  hyperinflation    emphysema          copd    bronchitis      markings     pneumonia  unfoldedaorta  chroniclungdisease       abscess  airbronchogram  aircrescent   airtrapping     apicalcap  aorticcalcification  atelectasis  azygosfissure     bluntedcp        border  bronchiectasis  bronchocele         bulla  bullousemphysema  calcification     carcinoma  cardiomegaly        cavity  consolidation          cyst    degenspine  diaphragmborder         edema      effusion      fibrosis  justfibrosis  fissuralshift  fissuralthickening      fracture   oldfracture   ribfracture  ribfractureall  claviclefracture  femurfracture    granuloma   groundglass      haziness   heartborder  hydropneumothorax  juxtaphrenicpeak        kerley        lesion  \

In [68]:
columns = list(df_tags.columns.values)
print(columns[224])

other
